In [1]:
import pandas as pd

In [2]:
df1 = pd.read_excel("Training.xlsx")
df2 = pd.read_excel("Validation.xlsx")

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
df1.drop(["연령", "성별", "신체질환", "상황키워드", "감정_소분류",  "시스템문장1", "시스템문장2", "시스템문장3"], axis=1, inplace=True)
df2.drop(["연령", "성별", "신체질환", "상황키워드", "감정_소분류",  "시스템문장1", "시스템문장2", "시스템문장3"], axis=1, inplace=True)

In [79]:
df1.rename(columns={"사람문장1" : "sentence"}, inplace=True)
df1.rename(columns={'감정_대분류': 'Emotion'}, inplace=True)

df2.rename(columns={"사람문장1" : "sentence"}, inplace=True)
df2.rename(columns={'감정_대분류': 'Emotion'}, inplace=True)


In [80]:
df_train = pd.concat([
    df1[['Emotion', 'sentence']],
    df1[['Emotion', '사람문장2']].rename(columns={'사람문장2': 'sentence'}),
    df1[['Emotion', '사람문장3']].rename(columns={'사람문장3': 'sentence'})
], ignore_index=True)

df_valid = pd.concat([
    df2[['Emotion', 'sentence']],
    df2[['Emotion', '사람문장2']].rename(columns={'사람문장2': 'sentence'}),
    df2[['Emotion', '사람문장3']].rename(columns={'사람문장3': 'sentence'})
], ignore_index=True)


In [84]:
df_train = df_train.dropna()
df_valid = df_valid.dropna()

In [88]:
df_train.to_csv("training_data.csv", index=False)
df_valid.to_csv("validation_data.csv", index=False)

In [36]:
import pandas as pd

In [37]:
df_train = pd.read_csv("training_data.csv")
df_valid = pd.read_csv("validation_data.csv")

In [38]:
from konlpy.tag import Okt

In [47]:
train = pd.read_csv("training_data.csv")

In [48]:
okt = Okt()

temp = []
for i in train['sentence']:
    temp.append(okt.morphs(i))
    
temp = pd.Series(temp)
train['token'] = temp

In [49]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145955 entries, 0 to 145954
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   Emotion   145955 non-null  object
 1   sentence  145955 non-null  object
 2   token     145955 non-null  object
dtypes: object(3)
memory usage: 3.3+ MB


In [50]:
train.to_csv("training_data-token.csv", index=False)

In [1]:
import pandas as pd

In [2]:
df_train = pd.read_csv("training_data-token.csv")

In [3]:
df_train['token']

0         ['일', '은', '왜', '해도', '해도', '끝', '이', '없을까', '...
1         ['이번', '달', '에', '또', '급여', '가', '깎였어', '!', '...
2         ['회사', '에', '신입', '이', '들어왔는데', '말투', '가', '거슬...
3         ['직장', '에서', '막내', '라는', '이유', '로', '나', '에게만'...
4         ['얼마', '전', '입사', '한', '신입사원', '이', '나를', '무시'...
                                ...                        
145950    ['노년', '층', '을', '위', '한', '경제', '적', '지원', '이...
145951    ['남편', '과', '함께', '게이트볼', '이나', '치러', '가야겠어', ...
145952    ['남편', '과', '함께', '실버', '일자리', '나', '노년', '층',...
145953    ['함께', '친하게', '지내던', '동네', '언니', '동생', '들', '과...
145954    ['사람', '들', '을', '볼', '때', '의심', '하고', '불신', '...
Name: token, Length: 145955, dtype: object

In [4]:
from gensim.models import Word2Vec
model = Word2Vec(df_train['token'], vector_size=1000, window=5, min_count=1, workers=4, sg=1)

KeyboardInterrupt: 

In [5]:
model.wv.vectors.shape

(1468, 1000)

In [8]:
import joblib

joblib.dump(model, 'word_model.joblib')

['word_model.joblib']

In [5]:
import pandas as pd
import numpy as np

In [6]:
def get_sent_embedding(model, embedding_size, tokenized_words):
    feature_vec = np.zeros((embedding_size,), dtype="float32")
    n_words = 0
    for word in tokenized_words:
        if word in model.wv.key_to_index:
            n_words += 1
            feature_vec = np.add(feature_vec, model.wv[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

In [8]:
from sklearn.svm import SVC
from joblib import load

model = load("word_model.joblib")

X = [get_sent_embedding(model, 1000, tokens) for tokens in df_train['token']]
y = df_train['Emotion']

# SVM 분류 모델 학습
svm_model = SVC(kernel='linear')
svm_model.fit(X, y)
joblib.dump(svm_model, 'svm_model.joblib')